# **Tên dự án** : Dữ đoán thông số sản xuất và thiết kế ngành In vải bằng máy học 
---
**1. Mô tả dự án:**
* Loại dự án - Phân loại
* Mô tả dự án:
    * Ngành công nghiệp in vải đang phát triển mạnh mẽ, với nhu cầu ngày càng cao về chất lượng in ấn, độ chính xác của màu sắc và hiệu suất sản xuất. Một trong những thách thức lớn nhất của ngành là việc xác định chính xác các thông số sản xuất (nhiệt độ, tốc độ ép, loại máy in, ...) và thông số thiết kế (mã màu, ICC profile, ...) để đảm bảo chất lượng đầu ra tối ưu.
    * Hiện nay, việc xác định các thông số này chủ yếu dựa vào kinh nghiệm của nhân viên kỹ thuật. Quy trình thủ công này có nhiều hạn chế:
        - Tốn thời gian: Các thông số cần được điều chỉnh thủ công, dẫn đến việc phải thử nghiệm nhiều lần.
        - Không nhất quán: Kinh nghiệm của mỗi nhân viên khác nhau, dẫn đến chất lượng in ấn không đồng đều.
        - Tỷ lệ lỗi cao: Sai lệch về màu sắc so với bản thiết kế gốc có thể xảy ra, gây lãng phí nguyên vật liệu.
    * Với sự phát triển của Trí tuệ nhân tạo (AI) và Học máy (Machine Learning), việc ứng dụng các thuật toán học máy vào dự đoán thông số sản xuất có thể giúp tự động hóa quy trình, tăng độ chính xác và giảm chi phí sản xuất.
    * Dự án FabricML: Dự đoán Thông Số Sản Xuất và Thiết Kế Ngành In Vải Bằng Máy Học được thực hiện nhằm giải quyết bài toán này, giúp tối ưu hóa quy trình sản xuất thông qua công nghệ học máy.
    
**2. Chi tiết chính**
* Tập dữ liệu bao gồm các thông tin sau:
    - LOẠI VẢI: Chất liệu vải (lụa Latin, thun lạnh, chiffon, gấm, v.v.)
    - CMYK ĐO: Giá trị màu đo được từ thiết bị (định dạng C-M-Y-K, ví dụ: 50-12-5-30)
    - CMYK FILE: Giá trị màu mục tiêu cần đạt được (định dạng tương tự)
    - MÁY IN: Loại máy in được sử dụng (ATEXCO-1, ATEXCO-2, MÁY 18, v.v.)
    - ICC: Hồ sơ màu (VEGAPRINT DEFAULT, PRINTFACTORY 2024, RIIN DEFAULT, v.v.)
    - MÁY ÉP: Thông số máy ép (ví dụ: 4.0, 5.0, 6.0)
    - TỐC ĐỘ: Tốc độ in (m/min)
    - HIỆT ĐỘ: Nhiệt độ ép (℃)
* Dữ liệu đầu vào:
    - Thông số CMYK đo được (CMYK ĐO): Gồm 4 giá trị C (Cyan), M (Magenta), Y (Yellow), K (Key/Black) được đo từ mẫu vải
    - Loại vải: Các loại vải khác nhau như LỤA LATIN, THUN LẠNH, CHIFON,...
    - Thông số máy móc: MÁY IN, ICC (hồ sơ màu), MÁY ÉP
* Mục tiêu dự đoán:
    - Thông số CMYK FILE (Thông số sản xuất lịch sử từ các lần in trước.): 4 giá trị C, M, Y, K của file thiết kế
    - Các thông số vận hành: TỐC ĐỘ, NHIỆT ĐỘ

## 1. Nhập thư viện 
---
Ở đây đang nhập tất cả các thư viện cần thiết.

In [2]:
# Nhập thư viện
# Nhập Numpy & Pandas để xử lý dữ liệu & sắp xếp dữ liệu
import numpy as np
import pandas as pd

# Nhập công cụ để trực quan hóa
import matplotlib.pyplot as plt
import seaborn as sns

# Nhập thư viện số liệu đánh giá
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report


# Thư viện được sử dụng để xử lý trước dữ liệu
from sklearn.preprocessing import LabelEncoder

# Nhập thư viện lựa chọn mô hình
from sklearn.model_selection import train_test_split, GridSearchCV

# Thư viện được sử dụng để triển khai Mô hình ML
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Thư viện được sử dụng để bỏ qua cảnh báo
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


## 2. Nhập bộ dữ liệu
---
Bây giờ hãy tải tập dữ liệu vào khung dữ liệu của gấu trúc. Để xử lý dữ liệu tốt hơn và tận dụng các chức năng tiện dụng để thực hiện các tác vụ phức tạp trong một lần.

In [6]:
# Tải tập dữ liệu
df = pd.read_excel("FAKE_DATA_1000.xlsx")

## 3. Chuẩn bị dữ liệu
---

In [ ]:
# Xem 5 hàng đầu tiên của tập dữ liệu
df.head()

In [ ]:
print("Số hàng: ", df.shape[0])
print("Số cột: ", df.shape[1])

In [ ]:
df.info()

In [ ]:
dup = df.duplicated().sum()
print(f'Số hàng trùng lặp: {dup}')

In [ ]:
df.isnull().sum()

## 4. Hiểu các biến
---

In [43]:
df.columns

Index(['LOẠI VẢI', 'CMYK ĐO', 'CMYK FILE', 'MÁY IN', 'ICC', 'MÁY ÉP', 'TỐC ĐỘ',
       'NHIỆT ĐỘ'],
      dtype='object')

In [41]:
df.describe(include='all').round(2)

,LOẠI VẢI,CMYK ĐO,CMYK FILE,MÁY IN,ICC,MÁY ÉP,TỐC ĐỘ,NHIỆT ĐỘ
count,1010,1010,1010,1010,1010,1010.00,1010.00,1010.0
unique,10,1010,1010,7,7,NaN,NaN,NaN
top,LỤA LATIN,50-12-5-30,48-10-7-32,ATEXCO-1,PRINTFACTORY DEFAULT,NaN,NaN,NaN
freq,119,1,1,513,511,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,5.86,11.94,225.0
std,NaN,NaN,NaN,NaN,NaN,0.62,4.07,0.0
min,NaN,NaN,NaN,NaN,NaN,1.00,10.00,225.0
25%,NaN,NaN,NaN,NaN,NaN,6.00,10.00,225.0
50%,NaN,NaN,NaN,NaN,NaN,6.00,11.00,225.0
75%,NaN,NaN,NaN,NaN,NaN,6.00,12.00,225.0


In [45]:
for i in df.columns.tolist():
    print("Số giá trị duy nhất trong", i, "là", df[i].nunique())

Số giá trị duy nhất trong LOẠI VẢI là 10
Số giá trị duy nhất trong CMYK ĐO là 1010
Số giá trị duy nhất trong CMYK FILE là 1010
Số giá trị duy nhất trong MÁY IN là 7
Số giá trị duy nhất trong ICC là 7
Số giá trị duy nhất trong MÁY ÉP là 6
Số giá trị duy nhất trong TỐC ĐỘ là 26
Số giá trị duy nhất trong NHIỆT ĐỘ là 1


## 4. Trực quan hóa dữ liệu ngành in vải
---
Dưới đây là các biểu đồ trực quan hóa quan trọng giúp phân tích dữ liệu in vải:


### 4.1 Phân bố các loại vải

Nhận xét: Dữ liệu có sự cân bằng tương đối giữa các loại vải, với Lụa Latin và Thun lạnh chiếm tỷ lệ cao nhất.

### 4.2. Biểu đồ nhiệt tương quan

Xem xét tương quan giữa các biến:

### 4.3 Giá trị thực tế vs dự đoán cho từng mô hình

Chúng ta sẽ sửa đổi vòng lặp chính để lưu trữ dự đoán và sau đó trực quan hóa chúng:

## 5. Xử lý dữ liệu
---

### 5.1 Tách giá trị CMYK từ chuỗi
---
Tách cột CMYK thành 4 cột riêng biệt

Cột CMYK ĐO và CMYK FILE chứa dữ liệu dạng chuỗi số, ví dụ "50-12-5-30". Chúng được tách thành 4 cột riêng biệt (C, M, Y, K) và chuyển đổi sang kiểu số (float).

In [8]:
df_cmyk_do = df["CMYK ĐO"].str.split('-', expand=True).astype(float)
df_cmyk_do.columns = ["CMYK ĐO_C", "CMYK ĐO_M", "CMYK ĐO_Y", "CMYK ĐO_K"]

df_cmyk_file = df["CMYK FILE"].str.split('-', expand=True).astype(float)
df_cmyk_file.columns = ["CMYK FILE_C", "CMYK FILE_M", "CMYK FILE_Y", "CMYK FILE_K"]


### 5.2 Mã hóa biến phân loại

* Các cột chứa dữ liệu dạng chữ (LOẠI VẢI, MÁY IN, ICC, MÁY ÉP) được chuyển thành số bằng LabelEncoder.

In [10]:
categorical_cols = ["LOẠI VẢI", "MÁY IN", "ICC", "MÁY ÉP"]
df_cat_transformed = df[categorical_cols].copy()
label_encoders = {} #  Khởi tạo dictionary giúp lưu lại bộ mã hóa

for col in categorical_cols:
    le = LabelEncoder()
    df_cat_transformed[col] = le.fit_transform(df_cat_transformed[col])# mã hoá dữ liệu
    label_encoders[col] = le  # Lưu lại encoder nếu cần dùng sau


* Chuyển đổi biến phân loại (LOẠI VẢI, MÁY IN, ICC, MÁY ÉP) sang dạng số bằng LabelEncoder().
* Điều này cần thiết vì mô hình XGBoost không xử lý dữ liệu dạng chuỗi.

### 5.3. Chuẩn hóa dữ liệu số
---
* Chuẩn hóa dữ liệu số bằng StandardScaler
* Các cột TỐC ĐỘ và NHIỆT ĐỘ được chuẩn hóa để dữ liệu có phân phối chuẩn.

In [13]:
numeric_cols = ["TỐC ĐỘ", "NHIỆT ĐỘ"]
df_num_transformed = df[numeric_cols].copy()
scaler_num = StandardScaler()
df_num_transformed[numeric_cols] = scaler_num.fit_transform(df_num_transformed[numeric_cols])

* Chuẩn hóa TỐC ĐỘ và NHIỆT ĐỘ để mô hình không bị ảnh hưởng bởi đơn vị đo lường.

### 5.4. Gộp tất cả lại thành DataFrame hoàn chỉnh

Các cột sau khi tiền xử lý được kết hợp lại (pd.concat), tạo ra tập dữ liệu sạch và sẵn sàng để đưa vào mô hình.

In [15]:
df_processed = pd.concat([df_cat_transformed, df_cmyk_do, df_cmyk_file, df_num_transformed], axis=1)
df_processed

,LOẠI VẢI,MÁY IN,ICC,MÁY ÉP,CMYK ĐO_C,CMYK ĐO_M,CMYK ĐO_Y,CMYK ĐO_K,CMYK FILE_C,CMYK FILE_M,CMYK FILE_Y,CMYK FILE_K,TỐC ĐỘ,NHIỆT ĐỘ
0,3,0,2,3,50.0,12.0,5.0,30.0,48.0,10.0,7.0,32.0,1.243870,0.0
1,6,1,6,4,0.0,12.0,30.0,5.0,2.0,10.0,32.0,7.0,3.946137,0.0
2,0,2,3,0,3.0,48.0,25.0,12.0,3.0,49.0,25.0,13.0,4.928779,0.0
3,4,6,5,4,0.0,35.0,36.0,3.0,0.0,35.0,35.0,3.0,4.437458,0.0
4,8,3,1,0,23.0,10.0,10.0,5.0,20.0,11.0,10.0,5.0,4.928779,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1,4,5,5,0.0,69.0,43.0,35.0,0.0,59.0,41.0,31.0,-0.475755,0.0
1006,1,1,6,5,34.0,97.0,46.0,7.0,31.0,83.0,44.0,7.0,0.506888,0.0
1007,2,0,3,5,69.0,64.0,52.0,72.0,64.0,55.0,50.0,65.0,0.015567,0.0
1008,6,0,3,5,100.0,73.0,73.0,57.0,92.0,63.0,70.0,51.0,-0.475755,0.0


* Gộp tất cả các dữ liệu đã xử lý để chuẩn bị cho việc huấn luyện mô hình.

## 6. Xây dựng mô hình
---

### 6.1 Chia dữ liệu thành đầu vào (X) và đầu ra (y)
* X: Các đặc trưng đầu vào như LOẠI VẢI, CMYK ĐO_C, CMYK ĐO_M, CMYK ĐO_Y, CMYK ĐO_K.
* y: Các giá trị cần dự đoán như CMYK FILE_C, CMYK FILE_M, CMYK FILE_Y, CMYK FILE_K, MÁY IN, ICC, MÁY ÉP, TỐC ĐỘ, NHIỆT ĐỘ.

In [17]:
input_columns = ["LOẠI VẢI", "CMYK ĐO_C", "CMYK ĐO_M", "CMYK ĐO_Y", "CMYK ĐO_K"]
target_columns = ["CMYK FILE_C", "CMYK FILE_M", "CMYK FILE_Y", "CMYK FILE_K", "MÁY IN", "ICC", "MÁY ÉP", "TỐC ĐỘ", "NHIỆT ĐỘ"]


In [19]:
# chia dữ liệu
X = df_processed[input_columns].copy()
y = df_processed[target_columns].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train.values
X_test = X_test.values


* Chia dữ liệu thành tập huấn luyện (70%) và tập kiểm tra (30%).

### 6.2 Danh sách mô hình huốn luyện 
* Ba mô hình Machine Learning được chọn:
    - Linear Regression: Mô hình tuyến tính đơn giản, dùng để kiểm tra mối quan hệ tuyến tính giữa các biến.
    - RandomForestRegressor: Mô hình học tập dựa trên nhiều cây quyết định, giúp dự đoán tốt hơn so với Linear Regression.
    - XGBoost: Một mô hình nâng cao, giúp cải thiện độ chính xác bằng cách giảm lỗi.
* Mỗi mô hình có bộ tham số (param_grid) dùng để tìm tham số tối ưu qua GridSearchCV.

#### 2.1 Huấn luyện mô hình XGBoost


In [27]:
model_xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
model_xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

* Sử dụng XGBRegressor, một mô hình mạnh mẽ cho bài toán hồi quy.
* objective='reg:squarederror' được dùng cho bài toán hồi quy.

#### 2.2 Huấn luyện mô hình RandomForestRegressor

In [23]:
model_RandomForest = RandomForestRegressor(random_state=42)
model_RandomForest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

#### 2.3 huấn luyện mô hình LinearRegression

In [25]:
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(X_train, y_train)

LinearRegression()

### 6.3 Phân tích hiệu suất mô hình

In [29]:
print("XGBoost Accuracy:", model_xgb.score(X_test, y_test) * 100)
print("RandomForest Accuracy:", model_RandomForest.score(X_test, y_test) * 100)
print("LinearRegression Accuracy:", model_LinearRegression.score(X_test, y_test) * 100)

XGBoost Accuracy: 70.41963934898376
RandomForest Accuracy: 67.59324824001776
LinearRegression Accuracy: 63.18015084385134


### 6.4 Đánh giá mô hình

In [36]:
models = {
    "XGBoost": model_xgb,
    "Random Forest": model_RandomForest,
    "Linear Regression": model_LinearRegression
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)  # Tính R-squared

    print(f"{name}:")
    print(f"  MAE: {mae}")
    print(f"  MSE: {mse}")
    print(f"  RMSE: {rmse}")
    print(f"  R²: {r2}")
    print("-" * 40)

XGBoost:
  MAE: 0.43143290281295776
  MSE: 0.8425724506378174
  RMSE: 0.9179174530630831
  R²: 0.7041963934898376
----------------------------------------
Random Forest:
  MAE: 2.1776775746100037
  MSE: 15.402122776649126
  RMSE: 3.924553831539214
  R²: 0.6759324824001776
----------------------------------------
Linear Regression:
  MAE: 0.6906521077812617
  MSE: 1.828158682753215
  RMSE: 1.3520941841281675
  R²: 0.6318015084385133
----------------------------------------


* Kết quả của từng mô hình được đánh giá bằng:
    - MAE (Mean Absolute Error): Càng nhỏ càng tốt.
    - MSE (Mean Squared Error) & RMSE: Đo lường sai số, RMSE dễ hiểu hơn vì cùng đơn vị với dữ liệu thực tế.
    - R² (R-squared): Đánh giá mức độ mô hình giải thích được dữ liệu.